This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [1]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = 'Hpnx3os6JxzB_L3HEhBa'

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules
import requests


In [15]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
url = 'https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json'
params = dict(api_key=API_KEY, start_date='2020-01-28', end_date='2020-01-28')
res = requests.get(url, params=params)

In [16]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
res.json()

{'dataset_data': {'collapse': None,
  'column_index': None,
  'column_names': ['Date',
   'Open',
   'High',
   'Low',
   'Close',
   'Change',
   'Traded Volume',
   'Turnover',
   'Last Price of the Day',
   'Daily Traded Units',
   'Daily Turnover'],
  'data': [['2020-01-28',
    118.8,
    119.6,
    115.2,
    118.1,
    None,
    134333.0,
    15754191.0,
    None,
    None,
    None]],
  'end_date': '2020-01-28',
  'frequency': 'daily',
  'limit': None,
  'order': None,
  'start_date': '2020-01-28',
  'transform': None}}

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [25]:
import sys

In [17]:
params = dict(api_key=API_KEY, start_date='2017-01-01', end_date='2017-12-31')
res = requests.get(url, params=params)

In [19]:
json = res.json()

In [46]:
def getMinOpen(d):
    m = float('inf')
    for i in range(len(d['dataset_data']['data'])):
        try:
            if d['dataset_data']['data'][i][1] < m:
                m = d['dataset_data']['data'][i][1]
        except: pass
    return m

def getMaxOpen(d):
    m = float('-inf')
    for i in range(len(d['dataset_data']['data'])):
        try:
            if d['dataset_data']['data'][i][1] > m:
                m = d['dataset_data']['data'][i][1]
        except: pass
    return m

In [48]:
print('highest open price: {}'.format(getMaxOpen(json)))
print('lowest open price: {}'.format(getMinOpen(json)))

highest open price: 53.11
lowest open price: 34.0


In [53]:
def getHighestChange(d):
    m = float('-inf')
    for i in range(len(d['dataset_data']['data'])):
        try:
            if abs(d['dataset_data']['data'][i][2] - d['dataset_data']['data'][i][3]) > m:
                m = abs(d['dataset_data']['data'][i][2] - d['dataset_data']['data'][i][3])
        except: pass
    return m

In [54]:
print('highest change in any one day: {}'.format(getHighestChange(json)))

highest change in any one day: 2.8100000000000023


In [59]:
def getHighestChangeTwo(d):
    m = float('-inf')
    for i in range(len(d['dataset_data']['data'])-1):
        try:
            if abs(d['dataset_data']['data'][i][4] - d['dataset_data']['data'][i+1][4]) > m:
                m = abs(d['dataset_data']['data'][i][4] - d['dataset_data']['data'][i+1][4])
        except: pass
    return m

In [60]:
print('highest change between any two days: {}'.format(getHighestChangeTwo(json)))

highest change between any two days: 2.559999999999995


In [62]:
def avgTradingVolume(d):
    m = 0
    count = 0
    for i in range(len(d['dataset_data']['data'])):
        try:
            m += d['dataset_data']['data'][i][6]
            count += 1
        except: pass
    return m / count

In [63]:
print('average daily trading volume: {}'.format(avgTradingVolume(json)))

average daily trading volume: 89124.33725490196


In [66]:
def medianTradingVolume(d):
    m = list()
    count = 0
    for i in range(len(d['dataset_data']['data'])):
        try:
            m.append(d['dataset_data']['data'][i][6])
            count += 1
        except: pass
    m.sort()
    if count % 2 == 0:
        return (m[count // 2] + m[(count // 2) + 1]) / 2
    else:
        return m[count // 2]

In [67]:
print('median trading volume: {}'.format(medianTradingVolume(json)))

median trading volume: 76286.0
